In [2]:
import pandas as pd
import os
import statsmodels.api as sm
import numpy as np

# Market Income Regression

In [3]:
current_dir = os.getcwd()
data_dir = os.path.dirname(current_dir)

file_path = os.path.join(data_dir, "Statistics Canada", "incomes_quintile_hh.xlsx")
df = pd.read_excel(file_path)

In [4]:
df["log_comp_all"] = np.log(df["comp_all"])
# Define independent (compx) and dependent (comp) variables
compx = df[["log_comp_all"]]  # Log of total income as independent variable

# Add a constant term (intercept)
compx = sm.add_constant(compx)

In [5]:
for i in range(1, 6):  
    quantile_col = f"comp_q{i}"
    log_quantile_col = f"log_comp_q{i}"
    
    if quantile_col not in df.columns:
        print(f"Column {quantile_col} not found in DataFrame.")
        continue  # Skip if the column is missing

    # Handle zero or negative values before log transformation
    if (df[quantile_col] <= 0).any():
        print(f"Skipping {quantile_col} due to zero/negative values.")
        continue

    # Compute log for the current quantile
    df[log_quantile_col] = np.log(df[quantile_col])

    # Define dependent variable (compy)
    compy = df[log_quantile_col]
    
    # Run the regression
    model = sm.OLS(compy, compx).fit()
    
    # Print results
    print(f"Quintile {i}")
    print(model.summary())
    print("\n" + "="*80 + "\n")


Quintile 1
                            OLS Regression Results                            
Dep. Variable:            log_comp_q1   R-squared:                       0.903
Model:                            OLS   Adj. R-squared:                  0.898
Method:                 Least Squares   F-statistic:                     213.1
Date:                Wed, 16 Apr 2025   Prob (F-statistic):           4.03e-13
Time:                        08:35:25   Log-Likelihood:                 39.626
No. Observations:                  25   AIC:                            -75.25
Df Residuals:                      23   BIC:                            -72.81
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.4599      0.592   

# Apply Growth Rates for 2022 & 2023

In [6]:
#calculate growth rates 

for i in range(1, 6):
    df[f"compq{i}_g"] = df[f"comp_q{i}"].pct_change()
    df[f"tranrq{i}_g"] = df[f"tranr_q{i}"].pct_change()


In [7]:
#store growth rates for 2022 and 2023 in a dictionary

vars = ['compq1', 'compq2', 'compq3', 'compq4', 'compq5', 'tranrq1', 'tranrq2', 'tranrq3', 'tranrq4', 'tranrq5']
growth_rates = {}  # Dictionary to store variable names and values

for var in vars:
    # Dynamically fetch values for 2022 and 2023 and assign them to keys in the dictionary
    growth_rates[f'canada_{var}_2022'] = df.loc[df['year'] == 2022, f'{var}_g'].values[0]
    growth_rates[f'canada_{var}_2023'] = df.loc[df['year'] == 2023, f'{var}_g'].values[0]

# Now print the stored results from the dictionary
for key, value in growth_rates.items():
    print(f'{key}: {value}')

canada_compq1_2022: 0.012403379471508158
canada_compq1_2023: 0.05779474431818188
canada_compq2_2022: 0.039472560020505254
canada_compq2_2023: 0.028740513438724413
canada_compq3_2022: 0.06814698983580914
canada_compq3_2023: 0.023233003454939283
canada_compq4_2022: 0.07878008173149698
canada_compq4_2023: 0.023342686776569588
canada_compq5_2022: 0.07499350214843847
canada_compq5_2023: 0.04678224188487068
canada_tranrq1_2022: -0.0039343480507093576
canada_tranrq1_2023: -0.006742738589211594
canada_tranrq2_2022: -0.02424912235079968
canada_tranrq2_2023: 0.004563928309680776
canada_tranrq3_2022: -0.07645217869793997
canada_tranrq3_2023: 0.04148357786549495
canada_tranrq4_2022: -0.09942419799005753
canada_tranrq4_2023: -0.012469719232754861
canada_tranrq5_2022: -0.11292128324904449
canada_tranrq5_2023: 0.06889198363652183


In [8]:
#import 2021 census, filter data
current_dir = os.getcwd()
data_dir = os.path.dirname(current_dir)

file_path_census = os.path.join(data_dir, "Statistics Canada", "Census 2021", "Hierarchical", "censush.csv")
census2021 = pd.read_csv(file_path_census)
census2021 = census2021[census2021["PR"] == 35]

# Drop rows where income equals 88888888 (dropping NA values)
census2021 = census2021[census2021["TOTINC"] != 88888888]
census2021 = census2021[census2021["MRKINC"] != 88888888]
census2021 = census2021[census2021["GTRFS"] != 88888888]
census2021 = census2021[census2021["TOTINC_AT"] != 88888888]

In [9]:
# List of columns to drop
wt_columns = [f"WT{i}" for i in range(1, 17)]

columns_to_drop = [
    "ABOID", "AGEIMM", "BFNMEMB", "BUILT", "CF_RP", "CFSTRUCT", "CIP2021", "CITIZEN", "CITOTH",
    "CONDO", "COW", "DIST", "DTYPE", "EF_RP", "EFCOVID_ERB", "EFDECILE", "EFDIMBM_2018", "EMPIN", 
    "ETHDER", "FCOND", "FOL", "FPTWK", "GENDER", "GENSTAT", "HDGREE", "HLMOSTEN", 
    "HLMOSTFR", "HLMOSTNO", "HLREGEN", "HLREGFR", "HLREGNO", "HRSWRK", "INCTAX", "JOBPERM", "KOL", 
   "LI_ELIG_OML_U18", "LICO_AT", "LICO_BT", "LOC_ST_RES", "LOCSTUD", "LOLIMA", "LOLIMB", 
    "LOMBM_2018", "LSTWRK", "LWMOSTEN", "LWMOSTFR", "LWMOSTNO", "LWREGEN", "LWREGFR", "LWREGNO", 
    "MARSTH", "MOB1", "MOB5", "MODE"
]

# Drop the columns
census2021 = census2021.drop(columns=wt_columns)
census2021 = census2021.drop(columns=columns_to_drop, errors="ignore")  # 'errors="ignore"' ensures no error if a column is missing


#set IMMSTAT = 1 if equal to 8
census2021["IMMSTAT"] = census2021["IMMSTAT"].replace(8, 1)

#create suitable bedroom variavle


# Step 1: Define Household-Level Calculation
def household_bedsuit(group):
    num_couples = (group['CFSTAT'] == 1).sum() // 2 + (group['CFSTAT'] == 2).sum() // 2 # Couples share a room
    num_single_parents = (group['CFSTAT'] == 3).sum()  # Each single parent gets a room
    num_children = (group['CFSTAT'].isin([4, 5])).sum()  # Count children
    num_non_family = (group['CFSTAT'].isin([7, 8])).sum()  # Each gets their own room
    num_living_alone = (group['CFSTAT'] == 6).sum()  # People living alone

    # If the household has exactly 1 person and they live alone → Assign bedsuit = 0 (bachelor unit)
    if len(group) == 1 and num_living_alone == 1:
        return 0  

    # Start with bedrooms for couples and single parents
    bedrooms_needed = num_couples + num_single_parents

    # Assign bedrooms for children: Every 2 children share 1 room
    if num_children > 0:
        bedrooms_needed += (num_children + 1) // 2  # Round up when odd number of children

    # Add rooms for non-family members (CFSTAT = 7, 8)
    bedrooms_needed += num_non_family

    # If NOS == 1, ensure bedsuit does NOT exceed BEDRM
    if 'NOS' in group.columns and group['NOS'].eq(1).any():  # Ensure 'NOS' exists
        max_bedrooms = group['BEDRM'].dropna().max()  # Get the max BEDRM in household
        if pd.notna(max_bedrooms):  
            bedrooms_needed = min(bedrooms_needed, max_bedrooms)  # Ensure it doesn't exceed BEDRM

    return bedrooms_needed

# Step 2: **Initialize `bedsuit` column with NaN**
census2021['bedsuit'] = pd.NA

# Step 3: Apply household-level logic
census2021['bedsuit'] = census2021.groupby('HH_ID')['bedsuit'].transform(
    lambda x: household_bedsuit(census2021.loc[x.index])
)

In [10]:
#create unemployment indicator
def assign_econ(row):
    if row['AGEGRP'] in [1, 2]:
        return 3
    elif 2 <= row['LFACT'] <= 10 and row['AGEGRP'] >= 3:
        return 1
    else:
        return 0

# Apply the function to create the ECON variable in census21
census2021['econ'] = census2021.apply(assign_econ, axis=1)


In [11]:
# Get the directory of the script (Model), then go up to Data
data_dir = os.path.dirname(os.getcwd())

# Build the full path to amr2021.xlsx
file_path = os.path.join(data_dir, "amr2021.xlsx")


#create a median market rent variable for Toronto/not-Toronto, by bedrooms for 2021
amr2021 = pd.read_excel(file_path)
# Perform a left join to add the MMR column to census2021
census2021 = census2021.merge(amr2021, on=['bedsuit', 'CMA'], how='left')

census2021['mmr'] = census2021['mmr'] 

In [12]:
# Step 1: Correct missing values (children with no income)
census2021.loc[census2021["TOTINC"] > 88000000, "TOTINC"] = 0
census2021.loc[census2021["MRKINC"] > 88000000, "MRKINC"] = 0
census2021.loc[census2021["GTRFS"] > 88000000, "GTRFS"] = 0
census2021.loc[census2021["TOTINC_AT"] > 88000000, "TOTINC_AT"] = 0

#check missing and NA values
variables = ["TOTINC", "MRKINC", "GTRFS"]
for var in variables:
    count = (census2021[var] > 88000000).sum()
    print(f"Number of records with {var} > 88000000: {count}")

census2021["totaltransfers"] = census2021["GTRFS"]

Number of records with TOTINC > 88000000: 0
Number of records with MRKINC > 88000000: 0
Number of records with GTRFS > 88000000: 0


create quintiles

In [13]:
# Step 1: Create a new column for total income
census2021["totalincome"] = census2021["MRKINC"] + census2021["GTRFS"]

# Step 2: Compute total household income (sum of MRKINC and GTRFS)
household_income = census2021.groupby("HH_ID").agg(
    totalincome=("totalincome", "sum"),  # Sum of the newly created totalincome
    weight=("WEIGHT", "first")  # Take first weight per household
).reset_index()


# Step 3: Compute weighted quintiles
def weighted_quantile(values, quantiles, sample_weight):
    """Calculate weighted quantiles for a given dataset."""
    sorter = np.argsort(values)
    values, sample_weight = values[sorter], sample_weight[sorter]
    weighted_cdf = np.cumsum(sample_weight) / np.sum(sample_weight)
    return np.interp(quantiles, weighted_cdf, values)

# Compute cutoff points for weighted quintiles
quantile_cutoffs = weighted_quantile(
    household_income["totalincome"].values,
    quantiles=[0.2, 0.4, 0.6, 0.8],
    sample_weight=household_income["weight"].values
)

# Step 4: Assign households to weighted quintiles
household_income["quintile"] = np.digitize(household_income["totalincome"], bins=quantile_cutoffs, right=True) + 1

# Step 5: Merge the quintile info back to the original dataset
census2021 = census2021.merge(household_income[["HH_ID", "quintile"]], on="HH_ID", how="left")

# Step 6: Verify the result
print(census2021[["HH_ID", "TOTINC", "quintile"]].head(10))

#subset census2021 for only quintiles 1 and 2
census2021 = census2021[census2021["quintile"].isin([1, 2, 3])]


   HH_ID  TOTINC  quintile
0      1   12000         1
1      4   34000         4
2      4   97000         4
3      4       0         4
4      4       0         4
5      4       0         4
6      6    3000         3
7      6    8000         3
8      6   69000         3
9      6       0         3


Identify transfer income shares using individual census file

In [14]:
current_dir = os.getcwd()
data_dir = os.path.dirname(current_dir)



# import individual census file.
file_path_censusi = os.path.join(data_dir, "Statistics Canada", "Census 2021", "data_donnees_2021_ind.csv")

census2021i = pd.read_csv(file_path_censusi)

census2021i = census2021i[census2021i["PR"] == 35]
census2021i = census2021i[census2021i["PRIHM"] == 1]
census2021i = census2021i[census2021i["HHInc"] <= 25]

#check how many rows have the different values of PRIHM
print(census2021i["PRIHM"].value_counts())

PRIHM
1    89550
Name: count, dtype: int64


In [15]:
# for columns HHInc,GTRfs, COVID_ERB, CQPPB, EICBN, OASGI, CHDBN, GovtI,  set values greater than 88000000 equal to 0
columns_to_fix = ["HHInc", "GTRfs", "COVID_ERB", "CQPPB", "EICBN", "OASGI", "CHDBN", "GovtI"]

# Identify rows where any of the specified columns has a value of 88888888
rows_to_drop = census2021i[census2021i[columns_to_fix].eq(88888888).any(axis=1)]

# Print how many rows will be dropped
print(f"Rows to be dropped: {len(rows_to_drop)}")

# Drop those rows from the DataFrame
census2021i = census2021i.drop(rows_to_drop.index)

# Optionally, confirm the rows are dropped (this will show the remaining rows)
print(f"Remaining rows: {len(census2021i)}")

# Replace 99999999 with 0 in the specified columns
census2021i[columns_to_fix] = census2021i[columns_to_fix].replace(99999999, 0)

# Verify if the replacement is successful
rows_to_check = census2021i[census2021i[columns_to_fix].eq(0).any(axis=1)]
print(f"Rows where 99999999 was replaced with 0: {len(rows_to_check)}")

#create a variable that is equal to GTRfs - COVID_ERB - CQPPB - EICBN - OASGI-CHDBN-Govtl
census2021i["other"] = census2021i["GTRfs"] - census2021i["COVID_ERB"] - census2021i["CQPPB"] - census2021i["EICBN"] - census2021i["OASGI"] - census2021i["CHDBN"]

# Display the head of census2021i with specified columns
print(census2021i[['PPSORT', 'HHInc', 'GTRfs', 'COVID_ERB', 'CQPPB', 'EICBN', 'OASGI', 'CHDBN', 'GovtI', 'other']].head())

# Define the columns to compute as share of GTRfs
share_columns = ["COVID_ERB", "CQPPB", "EICBN", "OASGI", "CHDBN", "other"]

# Calculate share of each column as a percentage of GTRfs
for column in share_columns:
    # Check if the column exists to avoid errors
    if column in census2021i.columns and "GTRfs" in census2021i.columns:
        census2021i[f"{column}_share"] = census2021i[column] / census2021i["GTRfs"]
    else:
        print(f"Column {column} or GTRfs not found in the dataframe")

# Print the new columns to verify
print(census2021i[["COVID_ERB_share", "CQPPB_share", "EICBN_share", "OASGI_share", "CHDBN_share", "other_share"]].head())

# Define income groups
low_income = census2021i["HHInc"] < 14
mid_income = (census2021i["HHInc"] >= 14) & (census2021i["HHInc"] <= 19)
high_income = census2021i["HHInc"] >19

# Function to compute weighted shares for a subset of the dataset and save them to individual variables
def compute_weighted_shares(df, label, prefix):
    weighted_sum_GTRfs = (df["GTRfs"] * df["WEIGHT"]).sum()
    
    for column in share_columns:
        if column in df.columns and "GTRfs" in df.columns:
            weighted_sum_column = (df[column] * df["WEIGHT"]).sum()
            share = weighted_sum_column / weighted_sum_GTRfs if weighted_sum_GTRfs != 0 else 0
            # Create individual variables and assign the computed share values
            globals()[f"{prefix}_{column}_share"] = share
        else:
            print(f"Column {column} or GTRfs not found in the dataframe for {label}")

# Compute for low-income group
compute_weighted_shares(census2021i[low_income], "HHInc < 44,000", "q1")

# Compute for mid-income group
compute_weighted_shares(census2021i[mid_income], "HHInc 44,001 - 74,000", "q2")

# Compute for high-income group
compute_weighted_shares(census2021i[high_income], "HHInc 74,000 - 110,000", "q3")

for column in share_columns:
    # Print variables for low-income group (q1)
    q1_share_value = globals().get(f'q1_{column}_share', 'Not Defined')
    print(f"q1_{column}_share: {q1_share_value}")
    
    # Print variables for mid-income group (q2)
    q2_share_value = globals().get(f'q2_{column}_share', 'Not Defined')
    print(f"q2_{column}_share: {q2_share_value}")

      # Print variables for high-income group (q3)
    q3_share_value = globals().get(f'q3_{column}_share', 'Not Defined')
    print(f"q3_{column}_share: {q3_share_value}")

# Sum all q1 shares excluding q1_COVID_ERB_share
q1_total_share = 0
for column in share_columns:
    if column != "COVID_ERB":  # Exclude q1_COVID_ERB_share
        q1_total_share += globals().get(f'q1_{column}_share', 0)

# Sum all q2 shares excluding q2_COVID_ERB_share
q2_total_share = 0
for column in share_columns:
    if column != "COVID_ERB":  # Exclude q2_COVID_ERB_share
        q2_total_share += globals().get(f'q2_{column}_share', 0)

# Sum all q3 shares excluding q3_COVID_ERB_share
q3_total_share = 0
for column in share_columns:
    if column != "COVID_ERB":  # Exclude q3_COVID_ERB_share
        q3_total_share += globals().get(f'q3_{column}_share', 0)


# Print the total sums for q1 and q2 and q3 (excluding COVID_ERB share)
print(f"Total q1 share (excluding COVID_ERB_share): {q1_total_share:.4f}")
print(f"Total q2 share (excluding COVID_ERB_share): {q2_total_share:.4f}")
print(f"Total q3 share (excluding COVID_ERB_share): {q3_total_share:.4f}")

# Define and normalize q1 and q2 share values
for column in share_columns:
    if column != "COVID_ERB":
        # Get the share values for q1 and q2 and q3
        q1_share = globals().get(f'q1_{column}_share', 0)
        q2_share = globals().get(f'q2_{column}_share', 0)
        q3_share = globals().get(f'q3_{column}_share', 0)
        
        # Calculate the total for q1 and q2 and q3 (excluding COVID_ERB share)
        q1_total_share = sum(globals().get(f'q1_{col}_share', 0) for col in share_columns if col != "COVID_ERB")
        q2_total_share = sum(globals().get(f'q2_{col}_share', 0) for col in share_columns if col != "COVID_ERB")
        q3_total_share = sum(globals().get(f'q3_{col}_share', 0) for col in share_columns if col != "COVID_ERB")
        
        # Normalize the share values
        q1_normalized_value = q1_share / q1_total_share if q1_total_share != 0 else 0
        q2_normalized_value = q2_share / q2_total_share if q2_total_share != 0 else 0
        q3_normalized_value = q3_share / q3_total_share if q3_total_share != 0 else 0
        
        # Store the normalized share values in the globals() dictionary
        globals()[f'q1_{column}_share_n'] = q1_normalized_value
        globals()[f'q2_{column}_share_n'] = q2_normalized_value
        globals()[f'q3_{column}_share_n'] = q3_normalized_value
        
        # Print the normalized share values
        print(f"q1_{column}_share_n: {q1_normalized_value:.4f}")
        print(f"q2_{column}_share_n: {q2_normalized_value:.4f}")
        print(f"q3_{column}_share_n: {q3_normalized_value:.4f}")

Rows to be dropped: 611
Remaining rows: 88939
Rows where 99999999 was replaced with 0: 88939
    PPSORT  HHInc  GTRfs  COVID_ERB  CQPPB  EICBN  OASGI  CHDBN  GovtI  other
8        9     25  13700      12100      0      0      0      0  13700   1600
19      20     13  22800          0  12900      0   8700      0   1100   1200
32      33     24  20100       8000      0   7300      0      0  12800   4800
59      60     15   7600          0      0      0      0   5000   2600   2600
74      75      8  17500          0      0      0      0      0  17500  17500
    COVID_ERB_share  CQPPB_share  EICBN_share  OASGI_share  CHDBN_share  \
8          0.883212     0.000000     0.000000     0.000000     0.000000   
19         0.000000     0.565789     0.000000     0.381579     0.000000   
32         0.398010     0.000000     0.363184     0.000000     0.000000   
59         0.000000     0.000000     0.000000     0.000000     0.657895   
74         0.000000     0.000000     0.000000     0.000000     0

Calculate 2021 Core Housing Need

In [16]:
# Step 1: Aggregate total household income
household_income2 = census2021.groupby('HH_ID')['totalincome'].sum().reset_index()

# Step 2: Merge back into census2021
census2021 = census2021.merge(household_income2, on='HH_ID', suffixes=('', '_hh'))


# identify households where every preson in unrelated or living alone
household_non_family = census2021.groupby('HH_ID')['CFSTAT'].apply(lambda x: set(x).issubset({6, 7})).reset_index()
household_non_family.columns = ['HH_ID', 'non_family_household']
household_non_family['non_family_household'] = household_non_family['non_family_household'].astype(int)

#merge back into census2021
census2021 = census2021.merge(household_non_family, on='HH_ID', how='left')

#Identify households where at least one maintainer (HHMAINP == 1) is aged 15-29 and attending school
maintainers = census2021[(census2021['HHMAINP'] == 1) & 
                         (census2021['AGEGRP'].isin([3, 4, 5])) & 
                         (census2021['ATTSCH'] == 1)]

excluded_households = maintainers['HH_ID'].unique()
census2021['student_household'] = census2021['HH_ID'].isin(excluded_households).astype(int) # 1 if student, 0 otherwise

# Step 3: Define Core Housing Need (CHN)
census2021['chn'] = 0  # Default to 0 (not in CHN)

# Condition: Housing is unaffordable OR unsuitable OR inadequate
housing_issue = (
    (census2021['SHELCO'] * 12 / census2021['totalincome_hh'] > 0.30) |  # Unaffordable
    (census2021['NOS'] == 0) |  # Unsuitable
    (census2021['REPAIR'] == 3)  # Inadequate
)

# Condition: Alternative market rent is also unaffordable
market_unaffordable = (census2021['mmr']) * 12 > 0.30 * census2021['totalincome_hh']

# Set Core Housing Need (CHN) variable, excluding non-family households with an eligible maintainer
census2021.loc[housing_issue & market_unaffordable & 
               ~((census2021['student_household'] == 1) & (census2021['non_family_household'] == 1)), 
               'chn'] = 1


# Create STIR (Shelter Cost to Income Ratio)
census2021["stir"] = (census2021["SHELCO"] * 12) / census2021["totalincome_hh"]

# Create ALTSTIR (Alternative STIR using AMR)
census2021["altstir"] = (census2021["mmr"] * 12) / census2021["totalincome_hh"]

# Update CHN: Exclude individuals with STIR >= 1
census2021.loc[census2021["stir"] >= 1, "chn"] = 0



# Step 4: Define Deep Core Housing Need (DCHN)
census2021['dchn'] = 0  # Default to 0

deep_housing_issue = (
    (census2021['SHELCO'] * 12 / census2021['totalincome_hh'] > 0.50) |  # Deeply unaffordable
    (census2021['NOS'] == 0) |  # Unsuitable
    (census2021['REPAIR'] == 3)  # Inadequate
)

deep_market_unaffordable = (census2021['mmr']) * 12 > 0.50 * census2021['totalincome_hh']

# Apply DCHN flag with same exclusions
census2021.loc[deep_housing_issue & deep_market_unaffordable &
               ~((census2021['student_household'] == 1) & (census2021['non_family_household'] == 1)), 
               'dchn'] = 1

# Update DCHN: Exclude individuals with STIR >= 1
census2021.loc[census2021["stir"] >= 1, "dchn"] = 0

In [17]:
census2022 = census2021.copy()

In [18]:


current_dir = os.getcwd()
data_dir = os.path.dirname(current_dir)

# Load the population projection file
file_path_pop = os.path.join(data_dir, "popproj.xlsx")
popproj = pd.read_excel(file_path_pop)

# Filter to 2022 growth rates only
growth_rates_2022 = popproj[['demo', 'agegrp', 'econ', 2022]]

# Create a lookup dictionary: {(agegrp, demo, econ): growth_rate}
growth_rate_lookup = {
    (row['agegrp'], row['demo'], row['econ']): row[2022]
    for _, row in growth_rates_2022.iterrows()
}

# Define the AGEGRP mapping from numbers to labels
agegrp_mapping = {
    1: '0to9',
    2: '10to14',
    3: '15to19',
    4: '20to24',
    5: '25to29',
    6: '30to34',
    7: '35to39',
    8: '40to44',
    9: '45to49',
    10: '50to54',
    11: '55to64',
    12: '65to74',
    13: '75plus',
    88: "total"
}

# Function to apply growth rates based on AGEGRP, IMMSTAT, and ECON
def apply_growth(row):
    agegrp_code = row['AGEGRP']
    immstat = row['IMMSTAT']
    econ = row['econ']

    # Map AGEGRP code to string label
    agegrp_label = agegrp_mapping.get(agegrp_code, None)
    
    if agegrp_label is None:
        print(f"Warning: Unknown AGEGRP code {agegrp_code} in row {row.name}")
        return row  # Skip updating this row

    # Determine if the person is NPR or non-NPR based on IMMSTAT
    demo = 'npr' if immstat == 3 else 'nonnpr'
    
    # Lookup the growth rate, defaulting to 1 if not found
    growth_rate = growth_rate_lookup.get((agegrp_label, demo, econ), 1)
    
    # Apply the growth rate
    row['WEIGHT'] *= (1 + growth_rate)
    return row

# Apply the updated growth function to census2022
census2022 = census2022.apply(apply_growth, axis=1)

summary statistics:
income quintile ranges
number of households per quintile
number of household in CHN per quintile

In [19]:
# Show the min and max household income for each quintile
print(household_income.groupby("quintile")["totalincome"].agg(["min", "max"]))

# Step 1: Aggregate weights at the household level
household_weights = census2021.groupby("HH_ID")["WEIGHT"].first().reset_index()

# Step 2: Merge back the quintile information
household_weights = household_weights.merge(
    household_income[["HH_ID", "quintile"]], on="HH_ID", how="left"
)

# Step 3: Compute weighted household counts by quintile
household_weighted_counts = household_weights.groupby("quintile")["WEIGHT"].sum()

# Print results with comma separators
print(household_weighted_counts.apply(lambda x: f"{x:,.0f}"))

# Sum total weights across all households
total_weight = household_weighted_counts.sum()
print(f"Total households (weighted): {total_weight:,.0f}")

             min      max
quintile                 
1         -38300    44300
2          44301    74500
3          74501   109900
4         110000   162101
5         162200  1613152
quintile
1    1,104,493
2    1,104,192
3    1,102,284
Name: WEIGHT, dtype: object
Total households (weighted): 3,310,969


In [20]:
# Keep one row per household in core housing need
core_housing_need_households = (
    census2021[census2021["HCORENEED_IND"] == 100]
    .drop_duplicates(subset="HH_ID")
)

# Sum of household weights by quintile
core_housing_need_weighted = (
    core_housing_need_households.groupby("quintile")["WEIGHT"]
    .sum()
)

# Format numbers with comma separators
print(core_housing_need_weighted.apply(lambda x: f"{x:,.0f}"))

#print sum of households in core housing need
total_core_housing_need = core_housing_need_weighted.sum()
print(total_core_housing_need)

quintile
1    467,645
2    166,994
3      5,824
Name: WEIGHT, dtype: object
640463.50217058


grow market income for 2022 based on Canada quintile data

In [21]:
def adjust_mrkinc(row):
    if row['quintile'] == 1:
        return row['MRKINC'] * (1 + growth_rates['canada_compq1_2022'])
    elif row['quintile'] == 2:
        return row['MRKINC'] * (1 + growth_rates['canada_compq2_2022'])
    elif row['quintile'] == 3:
        return row['MRKINC'] * (1 + growth_rates['canada_compq3_2022'])
    elif row['quintile'] == 4:
        return row['MRKINC'] * (1 + growth_rates['canada_compq4_2022'])
    elif row['quintile'] == 5:
        return row['MRKINC'] * (1 + growth_rates['canada_compq5_2022'])
    else:
        return row['MRKINC']  # Default case if quintile is missing

# Apply function row-wise
census2022.loc[:, 'MRKINC'] = census2022.apply(adjust_mrkinc, axis=1).astype('float64')

grow transfer income for 2022 based on Canada quintile data

In [22]:
def adjust_gtrfs(row):
    if row['quintile'] == 1:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq1_2022'])
    elif row['quintile'] == 2:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq2_2022'])
    elif row['quintile'] == 3:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq3_2022'])
    else:
        return row['GTRFS']  # Default case if quintile is missing

# Apply function row-wise
census2022.loc[:, 'GTRFS'] = census2022.apply(adjust_gtrfs, axis=1).astype('float64')

In [23]:
#new variable for total income (after growing MRKINC and GTRFS)
census2022["totalincome"] = census2022["MRKINC"] + census2022["GTRFS"]
census2022["totaltransfers"] = census2022["GTRFS"]

grow 2022 shelco based on tenur

In [24]:
print(census2022[['SHELCO', 'mmr']].head()) #check values before growing

   SHELCO     mmr
0   200.0   900.0
1   800.0  1700.0
2   800.0  1700.0
3   800.0  1700.0
4   800.0  1700.0


In [25]:
current_dir = os.getcwd()
data_dir = os.path.dirname(current_dir)


file_path_growth = os.path.join(data_dir, "growthrates.xlsx")
growth = pd.read_excel(file_path_growth)

print(growth.head())

#increase shelco variable by the 2022 rent variable in the growth df, if tenur is 2, increase by the mortgage variable if tenur is 1

year_to_use = 2022 

def adjust_shelco(row):
    rent = growth.loc[growth['year'] == year_to_use, 'rent'].values[0]
    mortgage = growth.loc[growth['year'] == year_to_use, 'mortgage'].values[0]
    othercosts = growth.loc[growth['year'] == year_to_use, 'othercosts'].values[0]

    if row['TENUR'] == 2:  # Renters
        return row['SHELCO'] * (1 + rent)
    elif row['TENUR'] == 1:
        if row['PRESMORTG'] in [1, 8]:  # Mortgage holders
            return row['SHELCO'] * (1 + mortgage)
        elif row['PRESMORTG'] == 0:  # No mortgage, use othercosts
            return row['SHELCO'] * (1 + othercosts)
    
    # Default case
    return row['SHELCO'] * (1 + rent)
#assigns unknown TENUR to renters

# Apply
census2022['SHELCO'] = census2022.apply(adjust_shelco, axis=1)
print(census2022[['SHELCO', 'mmr']].head())  # Check values after growing

   year      rent  mortgage  othercosts
0  2022  0.052632  0.080999    0.034682
1  2023  0.071429  0.118275    0.067737
2  2024  0.053333  0.039265    0.037933
3  2025  0.051074  0.027718    0.023945
4  2026  0.051079  0.036965    0.020308
       SHELCO     mmr
0  210.526316   900.0
1  827.745665  1700.0
2  827.745665  1700.0
3  827.745665  1700.0
4  827.745665  1700.0


In [26]:
#increase mmr variable by the 2022 rent variable in the growth df
census2022['mmr'] = census2022['mmr']*(1+growth.loc[growth['year'] == year_to_use, 'rent'].values[0])
print(census2022[['SHELCO', 'mmr']].head()) #check values after growing

       SHELCO          mmr
0  210.526316   947.368421
1  827.745665  1789.473684
2  827.745665  1789.473684
3  827.745665  1789.473684
4  827.745665  1789.473684


create 2023 census df, apply market and transfer income growth rates

In [27]:
# Creating 2023 microsimulation
census2023 = census2022.copy()

# Verify the result
print(census2023[["HH_ID", "PP_ID", "TOTINC", "quintile"]].head(10))

   HH_ID     PP_ID   TOTINC  quintile
0    1.0   11101.0  12000.0       1.0
1    6.0   61101.0   3000.0       3.0
2    6.0   61102.0   8000.0       3.0
3    6.0   61103.0  69000.0       3.0
4    6.0   61104.0      0.0       3.0
5   13.0  131101.0  63000.0       3.0
6   13.0  131102.0  41000.0       3.0
7   13.0  131103.0      0.0       3.0
8   13.0  131104.0      0.0       3.0
9   15.0  151101.0   5000.0       1.0


In [28]:
#adj weights for pop growth
# Filter to 2023 growth rates, now including 'econ'
growth_rates_2023 = popproj[['demo', 'agegrp', 'econ', 2023]]

# Create the 2023 growth rate lookup dictionary
growth_rate_lookup_2023 = {
    (row['agegrp'], row['demo'], row['econ']): row[2023]
    for _, row in growth_rates_2023.iterrows()
}

# Apply the 2023 growth rates
def apply_growth_2023(row):
    agegrp_code = row['AGEGRP']
    immstat = row['IMMSTAT']
    econ = row['econ']  # Include ECON in the logic

    agegrp_label = agegrp_mapping.get(agegrp_code, None)
    
    if agegrp_label is None:
        print(f"Warning: Unknown AGEGRP code {agegrp_code} in row {row.name}")
        return row  # Skip updating this row

    demo = 'npr' if immstat == 3 else 'nonnpr'
    
    # Lookup the growth rate, defaulting to 1 if not found
    growth_rate = growth_rate_lookup_2023.get((agegrp_label, demo, econ), 1)
    
    row['WEIGHT'] *= (1 + growth_rate)
    return row

# Apply the updated function to census2023
census2023 = census2023.apply(apply_growth_2023, axis=1)


In [29]:
def adjust_mrkinc(row):
    if row['quintile'] == 1:
        return row['MRKINC'] * (1 + growth_rates['canada_compq1_2023'])
    elif row['quintile'] == 2:
        return row['MRKINC'] * (1 + growth_rates['canada_compq2_2023'])
    elif row['quintile'] == 3:
        return row['MRKINC'] * (1 + growth_rates['canada_compq3_2023'])
    elif row['quintile'] == 4:
        return row['MRKINC'] * (1 + growth_rates['canada_compq4_2023'])
    elif row['quintile'] == 5:
        return row['MRKINC'] * (1 + growth_rates['canada_compq5_2023'])
    else:
        return row['MRKINC']  # Default case if quintile is missing

# Apply function row-wise
census2023.loc[:, 'MRKINC'] = census2023.apply(adjust_mrkinc, axis=1).astype('float64')


In [30]:
def adjust_gtrfs(row):
    if row['quintile'] == 1:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq1_2023'])
    elif row['quintile'] == 2:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq2_2023'])
    elif row['quintile'] == 3:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq3_2023'])
    elif row['quintile'] == 4:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq4_2023'])
    elif row['quintile'] == 5:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq5_2023'])
    else:
        return row['GTRFS']  # Default case if quintile is missing

# Apply function row-wise
census2023.loc[:, 'GTRFS'] = census2023.apply(adjust_gtrfs, axis=1).astype('float64')

In [31]:
#new variable for total income (after growing MRKINC and GTRFS)
census2023["totalincome"] = census2023["MRKINC"] + census2023["GTRFS"]
census2023["totaltransfers"] = census2023["GTRFS"]

For 2023 census, split out transfer income into subcategories, using shares calculated earlier

In [32]:
# Define the target columns and their corresponding share names
share_mapping = {
    "cpp": "CQPPB_share_n",
    "ei": "EICBN_share_n",
    "oas": "OASGI_share_n",
    "child": "CHDBN_share_n",
    "social": "other_share_n"
}

# Loop over quintiles 1 to 3
for quintile in [1, 2, 3]:
    for column, share_name in share_mapping.items():
        # Build the variable name, e.g., 'q1_CQPPB_share_n'
        variable_name = f'q{quintile}_{share_name}'
        share_value = globals().get(variable_name, None)
        
        if share_value is None:
            print(f"Error: {variable_name} is not defined.")
        else:
            census2023.loc[census2023["quintile"] == quintile, column] = share_value * census2023["GTRFS"]


In [33]:
#update shelco & mmr for 2023

year_to_use = 2023  

census2023['SHELCO'] = census2023.apply(adjust_shelco, axis=1)
census2023['mmr'] = census2023['mmr']*(1+growth.loc[growth['year'] == year_to_use, 'rent'].values[0])
print(census2023[['SHELCO', 'mmr']].head())  # Check values after growing

       SHELCO          mmr
0  225.563910  1015.037594
1  883.815029  1917.293233
2  883.815029  1917.293233
3  883.815029  1917.293233
4  883.815029  1917.293233


# Export Microsimulations for 2021-2023

In [34]:
#Export Census data (2021 to 2023) to .csv
folder_path = "../Microsimulations"
os.makedirs(folder_path, exist_ok=True)

census_years = [2021, 2022, 2023]

# Loop through each year and save the corresponding census dataset
for year in census_years:
    # Dynamically create the census variable name based on the year
    census = globals().get(f"census{year}", None)
    
    if census is not None:
        # Create the full path for saving the CSV file
        csv_filename = f"{folder_path}/census{year}.csv"  # Combine folder path and filename

        # Save the transformed dataset for the current year to a CSV file
        census.to_csv(csv_filename, index=False)  # Save to CSV (without the index)
        print(f"Saved census data for {year} to {csv_filename}")
    else:
        print(f"Dataset for {year} not found.")

Saved census data for 2021 to ../Microsimulations/census2021.csv
Saved census data for 2022 to ../Microsimulations/census2022.csv
Saved census data for 2023 to ../Microsimulations/census2023.csv


# 2024 Onwards Growth Rates and Microsimulations

In [35]:
forecast = pd.read_excel('..\\income_growthrates.xlsx')

#print all forecast df values
print(forecast)


# Creating global object that can be referenced later
for index, row in forecast.iterrows():
    globals()[row['share']] = row['value'] # Assign each row's value to a global variable with the corresponding name

           share     value
0   social_2024g  0.032989
1   social_2025g  0.019193
2   social_2026g  0.012367
3   social_2027g  0.010859
4   social_2028g  0.010948
5   social_2029g  0.010987
6   social_2030g  0.010941
7    child_2024g  0.077339
8    child_2025g  0.044419
9    child_2026g  0.009034
10   child_2027g  0.002045
11   child_2028g  0.004717
12   child_2029g  0.010578
13   child_2030g  0.012262
14     cpp_2024g  0.044154
15     cpp_2025g  0.026129
16     cpp_2026g  0.018968
17     cpp_2027g  0.018870
18     cpp_2028g  0.020122
19     cpp_2029g  0.020020
20     cpp_2030g  0.020108
21     oas_2024g  0.024188
22     oas_2025g  0.018645
23     oas_2026g  0.018914
24     oas_2027g  0.020359
25     oas_2028g  0.019953
26     oas_2029g  0.020138
27     oas_2030g  0.019741
28      ei_2024g  0.033365
29      ei_2025g  0.024689
30      ei_2026g  0.016714
31      ei_2027g  0.017807
32      ei_2028g  0.018187
33      ei_2029g  0.020394
34      ei_2030g  0.021054
35    comp_2024g  0.028474
3

In [36]:
popproj = pd.read_excel(file_path_pop)

In [37]:
# Create a lookup dictionary for each year's growth rates, considering ECON
growth_rate_lookups = {}

for year in range(2022, 2031):
    growth_rates_year = popproj[['demo', 'agegrp', 'econ', year]]
    growth_rate_lookups[year] = {
        (row['agegrp'], row['demo'], row['econ']): row[year]
        for _, row in growth_rates_year.iterrows()
    }

def apply_population_growth(row, year):
    agegrp_code = row['AGEGRP']
    immstat = row['IMMSTAT']
    econ = row['econ']  # Ensure ECON is included in calculations
    
    # Map AGEGRP code to string label
    agegrp_label = agegrp_mapping.get(agegrp_code, None)
    
    if agegrp_label is None:
        print(f"Warning: Unknown AGEGRP code {agegrp_code} in row {row.name}")
        return row  # Leave the row unchanged if AGEGRP is invalid

    demo = 'npr' if immstat == 3 else 'nonnpr'
    
    # Get the growth rate for the year, defaulting to 0% (no change) if missing
    growth_rate = growth_rate_lookups[year].get((agegrp_label, demo, econ), 0)
    
    # Apply the growth rate
    row['WEIGHT'] *= (1 + growth_rate)
    return row


In [38]:
# Define the years to loop over
years = range(2024, 2031)  # Covers 2024 to 2030

# Create a dictionary to store census data for each year
census= {"2023": census2023} #start with 2023 census data
weighted_sum_census = {}  # Dictionary to store weighted sum of MRKINC for each year

# Loop through each year
for year in years:
    # Initialize the new year's census data by copying the previous year's data
    prev_year = str(year - 1)
    curr_year = str(year)
    census[curr_year] = census[prev_year].copy()

    census[curr_year] = census[curr_year].apply(lambda row: apply_population_growth(row, year), axis=1)

    year_to_use = year
    census[curr_year]["SHELCO"] = census[curr_year].apply(adjust_shelco, axis=1)
    census[curr_year]['mmr'] = census[curr_year]['mmr']*(1+growth.loc[growth['year'] == year_to_use, 'rent'].values[0])

    # Grow transfer income
    census[curr_year]["ei"] *= (1 + globals()[f"ei_{curr_year}g"])
    census[curr_year]["child"] *= (1 + globals()[f"child_{curr_year}g"])
    census[curr_year]["social"] *= (1 + globals()[f"social_{curr_year}g"])
    census[curr_year]["cpp"] *= (1 + globals()[f"cpp_{curr_year}g"])
    census[curr_year]["oas"] *= (1 + globals()[f"oas_{curr_year}g"])

    # Compute total transfers for the year
    census[curr_year][f"totaltransfers"] = (
        census[curr_year]["ei"] +
        census[curr_year]["child"] +
        census[curr_year]["social"] +
        census[curr_year]["cpp"] +
        census[curr_year]["oas"]
    )

    # Grow market income by Canada-wide shares of total income growth for each quintile
    #canada_compq1 = 0.7834 * globals()[f"comp_{curr_year}g"]
    #canada_compq2 = 0.9956 * globals()[f"comp_{curr_year}g"]
    #canada_compq3 = 1.0034* globals()[f"comp_{curr_year}g"]

    canada_compq1 = 1 * globals()[f"comp_{curr_year}g"]
    canada_compq2 = 1 * globals()[f"comp_{curr_year}g"]
    canada_compq3 = 1* globals()[f"comp_{curr_year}g"]



    census[curr_year].loc[census[curr_year]["quintile"] == 1, "MRKINC"] *= (1 + canada_compq1)
    census[curr_year].loc[census[curr_year]["quintile"] == 2, "MRKINC"] *= (1 + canada_compq2)
    census[curr_year].loc[census[curr_year]["quintile"] == 3, "MRKINC"] *= (1 + canada_compq3)
    
    #create total income variable = transfer income + market income
    census[curr_year]["totalincome"] = census[curr_year]["MRKINC"] + census[curr_year]["totaltransfers"]

    # Create the full path for saving the CSV file
    csv_filename = f"{folder_path}/census{curr_year}.csv"  # Combine folder path and filename

    # Save the transformed dataset for the current year to a CSV file
    census[curr_year].to_csv(csv_filename, index=False)  # Save to CSV (without the index)
    print(f"Saved census data for {curr_year} to {csv_filename}")

Saved census data for 2024 to ../Microsimulations/census2024.csv
Saved census data for 2025 to ../Microsimulations/census2025.csv
Saved census data for 2026 to ../Microsimulations/census2026.csv
Saved census data for 2027 to ../Microsimulations/census2027.csv
Saved census data for 2028 to ../Microsimulations/census2028.csv
Saved census data for 2029 to ../Microsimulations/census2029.csv
Saved census data for 2030 to ../Microsimulations/census2030.csv


In [39]:
#check weights were applied properly for first record


#print the variables IMMSTAT, AGEGRP, WEIGHT for the first each record in each censusdf then calculate the percent change in the printed WEIGHT value
for year, df in census.items():
    print(f"Year: {year}")
    print(df[['IMMSTAT', 'AGEGRP', 'WEIGHT']].head(1))

#create an array out of each printed WEIGHT value from the previous loop
weights = [df['WEIGHT'].values[0] for df in census.values()]


#print weights array
print(weights)

#calculate the percent change in the values in the weights array
percent_change = [(weights[i] - weights[i - 1]) / weights[i - 1] * 100 for i in range(1, len(weights))]
print(percent_change)

Year: 2023
   IMMSTAT  AGEGRP      WEIGHT
0      1.0    11.0  100.534264
Year: 2024
   IMMSTAT  AGEGRP     WEIGHT
0      1.0    11.0  98.114029
Year: 2025
   IMMSTAT  AGEGRP  WEIGHT
0      1.0    11.0  98.377
Year: 2026
   IMMSTAT  AGEGRP     WEIGHT
0      1.0    11.0  97.838664
Year: 2027
   IMMSTAT  AGEGRP     WEIGHT
0      1.0    11.0  96.929779
Year: 2028
   IMMSTAT  AGEGRP     WEIGHT
0      1.0    11.0  95.538846
Year: 2029
   IMMSTAT  AGEGRP     WEIGHT
0      1.0    11.0  93.814007
Year: 2030
   IMMSTAT  AGEGRP     WEIGHT
0      1.0    11.0  92.438128
[np.float64(100.53426364451096), np.float64(98.11402910371264), np.float64(98.37699986746418), np.float64(97.83866366116509), np.float64(96.92977895273891), np.float64(95.53884635781219), np.float64(93.81400667795971), np.float64(92.43812812091856)]
[np.float64(-2.407372823017101), np.float64(0.2680256494956119), np.float64(-0.5472175478255608), np.float64(-0.9289627172074034), np.float64(-1.4349899586637003), np.float64(-1.80538047

create trace file

In [40]:
# Define the years you want to include
years = range(2021, 2031)

# List to hold second rows
second_rows = []

for year in years:
    file_path = f"../Microsimulations/census{year}.csv"
    
    try:
        # Read the full CSV and select the second row (index 1)
        second_row = pd.read_csv(file_path).iloc[[0]].copy()
        
        # Add a 'year' column
        second_row['year'] = year
        
        # Append to the list
        second_rows.append(second_row)
    
    except FileNotFoundError:
        print(f"File not found for year {year}: {file_path}")
        continue
    except IndexError:
        print(f"File {file_path} does not have a second row.")
        continue

# Combine all second rows into one DataFrame
summary_df = pd.concat(second_rows, ignore_index=True)

# Define the columns you want to keep
columns_to_keep = [
    'year', 'HH_ID', 'PP_ID', 'SHELCO', 'TENUR', 'PRESMORTG', 'AGEGRP' 'quintile', 
    'totalincome', 'totaltransfers', 'social', 'cpp', 
    'ei', 'oas', 'child', 'MRKINC', 'GTRFS'
]

# Keep only the desired columns (ignore missing ones)
summary_df = summary_df[[col for col in columns_to_keep if col in summary_df.columns]]

# Set 'year' as the index
summary_df.set_index('year', inplace=True)

# Transpose the DataFrame
summary_transposed = summary_df.transpose()

# Save the transposed DataFrame to Excel
output_path = "../Microsimulations/census_trace.xlsx"
summary_transposed.to_excel(output_path)

print(f"Created transposed summary Excel file with years as columns at {output_path}")


Created transposed summary Excel file with years as columns at ../Microsimulations/census_trace.xlsx


In [41]:


# Define input and output folders
input_folder = "../Microsimulations"
output_folder = "../Microsimulations/household"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Define years for processing
years = range(2021, 2031)  # Covers 2023 to 2030

# Loop through each year's census simulation file
for year in years:
    input_file = f"{input_folder}/census{year}.csv"
    output_file = f"{output_folder}/census{year}_household.csv"
    
    if not os.path.exists(input_file):
        print(f"Skipping {year}: File not found -> {input_file}")
        continue

    # Load individual-level census data
    df = pd.read_csv(input_file)

    # Compute average WEIGHT for adults (AGEGRP > 2)
    adult_weights = df[df["AGEGRP"] > 2].groupby("HH_ID")["WEIGHT"].mean().reset_index()
    adult_weights.rename(columns={"WEIGHT": "avg_adult_weight"}, inplace=True)

    # Group by HH_ID and aggregate
    household_df = df.groupby("HH_ID").agg({
        "BEDRM": "first",
        "HCORENEED_IND": "first",
        "REPAIR": "first",
        "NOS": "first",
        "PRESMORTG": "first",
        "SHELCO": "first",
        "TENUR": "first",
        "SUBSIDY": "first",
        "quintile": "first",
        "totalincome": "sum",
        "MRKINC": "sum",
        "TOTINC_AT": "sum",
        "totaltransfers": "sum",
        "mmr":"first",
        "student_household": "first",
        "non_family_household": "first",
        "chn":"first",
         "dchn":"first",
        "bedsuit":"first",
        "stir" : "first"
    }).reset_index()

   # Merge average adult weight into the household-level dataframe
    household_df = household_df.merge(adult_weights, on="HH_ID", how="left")

   # Rename the merged column to WEIGHT
    household_df.rename(columns={"avg_adult_weight": "WEIGHT"}, inplace=True)

    # Save the transformed household-level data
    household_df.to_csv(output_file, index=False)

    ###To do: gross up weights to align with FAO household projection###
    
    print(f"Saved household-level simulation for {year} -> {output_file}")





Saved household-level simulation for 2021 -> ../Microsimulations/household/census2021_household.csv
Saved household-level simulation for 2022 -> ../Microsimulations/household/census2022_household.csv
Saved household-level simulation for 2023 -> ../Microsimulations/household/census2023_household.csv
Saved household-level simulation for 2024 -> ../Microsimulations/household/census2024_household.csv
Saved household-level simulation for 2025 -> ../Microsimulations/household/census2025_household.csv
Saved household-level simulation for 2026 -> ../Microsimulations/household/census2026_household.csv
Saved household-level simulation for 2027 -> ../Microsimulations/household/census2027_household.csv
Saved household-level simulation for 2028 -> ../Microsimulations/household/census2028_household.csv
Saved household-level simulation for 2029 -> ../Microsimulations/household/census2029_household.csv
Saved household-level simulation for 2030 -> ../Microsimulations/household/census2030_household.csv


In [42]:
# Define the years you want to include
years = range(2021, 2031)

# List to hold second rows
second_rows = []

for year in years:
    file_path = f"../Microsimulations/household/census{year}_household.csv"
    
    try:
        # Read the full CSV and select the second row (index 1)
        second_row = pd.read_csv(file_path).iloc[[0]].copy()
        
        # Add a 'year' column
        second_row['year'] = year
        
        # Append to the list
        second_rows.append(second_row)
    
    except FileNotFoundError:
        print(f"File not found for year {year}: {file_path}")
        continue
    except IndexError:
        print(f"File {file_path} does not have a second row.")
        continue

# Combine all second rows into one DataFrame
summary_df = pd.concat(second_rows, ignore_index=True)

# Define the columns you want to keep
columns_to_keep = [
    'year', 'HH_ID','SHELCO', 'NOS', 'REPAIR', 'BEDRM', 'TENUR', 'PRESMORTG', 'AGEGRP' 'quintile', 
    'totalincome', 'totaltransfers', 'MRKINC', 'GTRFS'
]

# Keep only the desired columns (ignore missing ones)
summary_df = summary_df[[col for col in columns_to_keep if col in summary_df.columns]]

# Set 'year' as the index
summary_df.set_index('year', inplace=True)

# Transpose the DataFrame
summary_transposed = summary_df.transpose()

# Save the transposed DataFrame to Excel
output_path = "../Microsimulations/household/census_trace_hh.xlsx"
summary_transposed.to_excel(output_path)

print(f"Created transposed summary Excel file with years as columns at {output_path}")


Created transposed summary Excel file with years as columns at ../Microsimulations/household/census_trace_hh.xlsx


In [43]:


# Define the years and quintiles
years = range(2021, 2031)
quintiles = [1, 2, 3]

# Dictionaries to store weighted sums and weighted counts by year and quintile
weighted_HH_MRKINC = {year: {} for year in years}
weighted_HH_transfers = {year: {} for year in years}
weighted_HH_totalincome = {year: {} for year in years}  # NEW for totalincome
weighted_HH_counts = {year: {} for year in years}  # To store sum of weights per quintile

# Loop through each year and calculate weighted sums & counts by quintile
for year in years:
    file_path = f"../Microsimulations/household/census{year}_household.csv"
    
    try:
        df = pd.read_csv(file_path)
        
        # Ensure household-level aggregation by taking the first value per HH_ID
        df_household = df.groupby("HH_ID").agg(
            {
                "MRKINC": "first",
                "totaltransfers": "first",
                "totalincome": "first",  # NEW: Add totalincome
                "WEIGHT": "first",
                "quintile": "first"
            }
        ).reset_index()

        for q in quintiles:
            df_q = df_household[df_household["quintile"] == q]
            
            # Calculate weighted sums
            weighted_HH_MRKINC[year][q] = (df_q["MRKINC"] * df_q["WEIGHT"]).sum()
            weighted_HH_transfers[year][q] = (df_q["totaltransfers"] * df_q["WEIGHT"]).sum()
            weighted_HH_totalincome[year][q] = (df_q["totalincome"] * df_q["WEIGHT"]).sum()  # NEW: Total income
            
            # Calculate total weight (sum of weights) per quintile
            weighted_HH_counts[year][q] = df_q["WEIGHT"].sum()
    
    except FileNotFoundError:
        print(f"File not found for year {year}: {file_path}")
        continue

# Compute per-household weighted averages
avg_HH_MRKINC = {
    year: {q: (weighted_HH_MRKINC[year][q] / weighted_HH_counts[year][q]) if weighted_HH_counts[year][q] != 0 else None for q in quintiles}
    for year in years
}
avg_HH_transfers = {
    year: {q: (weighted_HH_transfers[year][q] / weighted_HH_counts[year][q]) if weighted_HH_counts[year][q] != 0 else None for q in quintiles}
    for year in years
}
avg_HH_totalincome = {  # NEW: Compute avg for total income
    year: {q: (weighted_HH_totalincome[year][q] / weighted_HH_counts[year][q]) if weighted_HH_counts[year][q] != 0 else None for q in quintiles}
    for year in years
}

# Print Year-over-Year Growth for Household Market Income (MRKINC)
print("\n--- Year-over-Year Per-Household MRKINC Growth by Quintile ---")
for q in quintiles:
    print(f"\nQuintile {q}:")
    for year in range(2022, 2031):  # Start from 2023 since we compare against the previous year
        prev = avg_HH_MRKINC.get(year - 1, {}).get(q)
        curr = avg_HH_MRKINC.get(year, {}).get(q)
        growth = ((curr - prev) / prev) * 100 if prev is not None and prev != 0 else None
        print(f"Year {year} | Avg HH MRKINC YoY Growth: {growth:.2f}%" if growth is not None else f"Year {year} | Avg HH MRKINC YoY Growth: N/A")

# Print Year-over-Year Growth for Household Total Transfers
print("\n--- Year-over-Year Per-Household Total Transfers Growth by Quintile ---")
for q in quintiles:
    print(f"\nQuintile {q}:")
    for year in range(2022, 2031):
        prev = avg_HH_transfers.get(year - 1, {}).get(q)
        curr = avg_HH_transfers.get(year, {}).get(q)
        growth = ((curr - prev) / prev) * 100 if prev is not None and prev != 0 else None
        print(f"Year {year} | Avg HH Transfers YoY Growth: {growth:.2f}%" if growth is not None else f"Year {year} | Avg HH Transfers YoY Growth: N/A")

# Print Year-over-Year Growth for Household Total Income
print("\n--- Year-over-Year Per-Household Total Income Growth by Quintile ---")
for q in quintiles:
    print(f"\nQuintile {q}:")
    for year in range(2022, 2031):
        prev = avg_HH_totalincome.get(year - 1, {}).get(q)
        curr = avg_HH_totalincome.get(year, {}).get(q)
        growth = ((curr - prev) / prev) * 100 if prev is not None and prev != 0 else None
        print(f"Year {year} | Avg HH Total Income YoY Growth: {growth:.2f}%" if growth is not None else f"Year {year} | Avg HH Total Income YoY Growth: N/A")



--- Year-over-Year Per-Household MRKINC Growth by Quintile ---

Quintile 1:
Year 2022 | Avg HH MRKINC YoY Growth: 1.07%
Year 2023 | Avg HH MRKINC YoY Growth: 6.12%
Year 2024 | Avg HH MRKINC YoY Growth: 3.10%
Year 2025 | Avg HH MRKINC YoY Growth: 3.15%
Year 2026 | Avg HH MRKINC YoY Growth: 3.16%
Year 2027 | Avg HH MRKINC YoY Growth: 3.23%
Year 2028 | Avg HH MRKINC YoY Growth: 2.62%
Year 2029 | Avg HH MRKINC YoY Growth: 2.56%
Year 2030 | Avg HH MRKINC YoY Growth: 2.64%

Quintile 2:
Year 2022 | Avg HH MRKINC YoY Growth: 4.32%
Year 2023 | Avg HH MRKINC YoY Growth: 2.82%
Year 2024 | Avg HH MRKINC YoY Growth: 2.72%
Year 2025 | Avg HH MRKINC YoY Growth: 3.50%
Year 2026 | Avg HH MRKINC YoY Growth: 3.39%
Year 2027 | Avg HH MRKINC YoY Growth: 3.43%
Year 2028 | Avg HH MRKINC YoY Growth: 2.76%
Year 2029 | Avg HH MRKINC YoY Growth: 2.61%
Year 2030 | Avg HH MRKINC YoY Growth: 2.66%

Quintile 3:
Year 2022 | Avg HH MRKINC YoY Growth: 7.06%
Year 2023 | Avg HH MRKINC YoY Growth: 2.23%
Year 2024 | Avg H

In [44]:
# Define the years and benefits of interest
years = [2023, 2024, 2025]
benefits = ["social", "cpp", "ei", "oas", "child", "totaltransfers"]

# Dictionary to store weighted sums and counts per year
weighted_sums = {year: {b: 0 for b in benefits} for year in years}
total_weights = {year: 0 for year in years}

# Process each year's data
for year in years:
    file_path = f"../Microsimulations/census{year}.csv"
    
    try:
        df = pd.read_csv(file_path)
        
        # Group by household (HH_ID) and aggregate:
        df_household = df.groupby("HH_ID").agg(
            {b: "sum" for b in benefits} | {"WEIGHT": "first", "quintile": "first"}  # Sum benefits, keep first weight & quintile
        ).reset_index()
        
        # Filter for Quintile 1
        df_q1 = df_household[df_household["quintile"] == 1]
        
        # Compute weighted sums for each benefit
        for b in benefits:
            weighted_sums[year][b] = (df_q1[b] * df_q1["WEIGHT"]).sum()
        
        # Compute total weight for Quintile 1
        total_weights[year] = df_q1["WEIGHT"].sum()
    
    except FileNotFoundError:
        print(f"File not found for year {year}: {file_path}")
        continue

# Compute weighted averages
weighted_averages = {
    year: {b: (weighted_sums[year][b] / total_weights[year]) if total_weights[year] > 0 else None for b in benefits}
    for year in years
}

# Print results
print("\n--- Weighted Average Household Benefit Values for Quintile 1 ---")
for year in years:
    print(f"\nYear {year}:")
    for b in benefits:
        value = weighted_averages[year][b]
        print(f"  {b.capitalize()} Benefit: {value:,.2f}" if value is not None else f"  {b.capitalize()} Benefit: N/A")


--- Weighted Average Household Benefit Values for Quintile 1 ---

Year 2023:
  Social Benefit: 4,971.47
  Cpp Benefit: 4,794.62
  Ei Benefit: 621.95
  Oas Benefit: 5,997.02
  Child Benefit: 890.53
  Totaltransfers Benefit: 17,275.59

Year 2024:
  Social Benefit: 5,074.91
  Cpp Benefit: 4,947.28
  Ei Benefit: 635.12
  Oas Benefit: 6,069.65
  Child Benefit: 948.08
  Totaltransfers Benefit: 17,675.04

Year 2025:
  Social Benefit: 5,229.49
  Cpp Benefit: 5,132.67
  Ei Benefit: 658.00
  Oas Benefit: 6,251.17
  Child Benefit: 1,001.14
  Totaltransfers Benefit: 18,272.47
